In [ ]:
# Load the geotiff
%matplotlib widget
# Now, note that our .tiff image is actually a GEOTIFF. This means that it has geographic information built-in
import cartopy.crs as ccrs
import rasterio
import numpy as np 
# filename = 'LT05_L1TP_153042_19990321_20200908_02_T1_refl.tif'
base_filename = 'landsat_images/LT05_L1TP_153043_19990406_20200908_02_T1_'
r_filename = base_filename + 'B4.TIF';
g_filename = base_filename + 'B3.TIF';
b_filename = base_filename + 'B2.TIF';

r_raster = rasterio.open(r_filename,'r') # open for reading
r_landsat = r_raster.read() # read the image from the rasterio context
g_raster = rasterio.open(g_filename,'r') # open for reading
g_landsat = g_raster.read() # read the image from the rasterio context
b_raster = rasterio.open(b_filename,'r') # open for reading
b_landsat = b_raster.read() # read the image from the rasterio context

print(r_landsat.shape)
# Note - the landsat image is 3x(ny)x(nx). Reshape the image to be (ny)x(nx)x3 via transpose:
landsat = r_landsat.transpose((1,2,0))
# Print out the meta-data. This contains information about the value used to fill missing data and some other things.
print(r_raster.meta)

In [ ]:
test=np.concatenate((r_landsat,g_landsat,b_landsat))
test.shape
landsat = test.transpose((1,2,0))

np.max(b_landsat[:])

# Coordinate system

Based on the information in the metadata for the image and information on epsg.io:

EPSG:32642

WGS 84 / UTM zone 42N

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(landsat)
plt.show()

In [ ]:
# Get more information about the coordinates
raster = r_raster

upper_left = raster.transform * (0,0)
lower_right = raster.transform * (raster.width,raster.height)

upper_right = raster.transform * (raster.width,0)
lower_left = raster.transform * (0,raster.height)

print(upper_left)
print(lower_right)
xmin=upper_left[0]
xmax=lower_right[0]
ymin=lower_right[1]
ymax=upper_left[1]

plt.figure()
plt.plot(upper_left[0],upper_left[1],'.')
plt.plot(upper_right[0],upper_right[1],'.')
plt.plot(lower_left[0],lower_left[1],'.')
plt.plot(lower_right[0],lower_right[1],'.')
plt.title('Coordinates of corners of image, in UTM')
plt.xlabel('Easting (m)')
plt.ylabel('Northing (m)')
plt.show()

In [ ]:
import numpy as np
x_values = np.linspace(lower_left[0],lower_right[0],raster.width)
y_values = np.linspace(upper_left[1],lower_right[1],raster.height)
%matplotlib widget

fig,ax = plt.subplots(1,1)
cm = plt.get_cmap('terrain')
h=ax.pcolorfast(x_values,y_values,landsat,cmap=cm)

ax.set_xlabel('Easting (m)')
ax.set_ylabel('Northing (m)')
ax.set_aspect('equal','box')
plt.show()

In [ ]:
def read_landsat(base_filename):
    # INPUT should be the filename up to the T1
    # OUTPUT will be x,y,image
    # where x is the x coordinates in m east
    # y is the y coordinates in m north
    # image is (ny x nx x 3) matrix containing the landsat scene
    bands = (4,3,2)
    r_filename = base_filename + 'B{:d}.TIF'.format(bands[0]);
    g_filename = base_filename + 'B{:d}.TIF'.format(bands[1]);
    b_filename = base_filename + 'B{:d}.TIF'.format(bands[2]);
    # Load R,G,B, bands from separate files
    r_raster = rasterio.open(r_filename,'r') # open for reading
    r_landsat = r_raster.read() # read the image from the rasterio context
    g_raster = rasterio.open(g_filename,'r') # open for reading
    g_landsat = g_raster.read() # read the image from the rasterio context
    b_raster = rasterio.open(b_filename,'r') # open for reading
    b_landsat = b_raster.read() # read the image from the rasterio context
    # Combine the three separate bands into a sigle image
    landsat = np.concatenate((r_landsat,g_landsat,b_landsat))
    landsat = landsat.transpose((1,2,0))
    # Get coordinate information:
    raster = r_raster
    upper_left = raster.transform * (0,0)
    lower_right = raster.transform * (raster.width,raster.height)
    upper_right = raster.transform * (raster.width,0)
    lower_left = raster.transform * (0,raster.height)
    x_values = np.linspace(lower_left[0],lower_right[0],raster.width)
    y_values = np.linspace(upper_left[1],lower_right[1],raster.height)
    # return x, y, and the image
    return x_values, y_values, landsat

def subset_from_coordinates(x,y,image,new_x_range,new_y_range):
    maskx = np.where(np.logical_and(x >= new_x_range[0],x <= new_x_range[1]))[0]
    masky = np.where(np.logical_and(y >= new_y_range[0],y <= new_y_range[1]))[0]
    new_x = x[maskx]
    new_y = y[masky]
    print(maskx)
    new_image = image[masky,:,:]
    new_image = new_image[:,maskx,:]
    return new_x,new_y,new_image
    
base_filename = 'landsat_images/LT05_L1TP_153043_19990406_20200908_02_T1_'
x,y,image = read_landsat(base_filename)
%matplotlib inline
fig,ax = plt.subplots(1,1)
cm = plt.get_cmap('terrain')
h=ax.pcolorfast(x,y,image,cmap=cm)
ax.set_xlim((760000,774000))
ax.set_ylim((2802000,2807000))
ax.set_aspect('equal')

newx,newy,newim = subset_from_coordinates(x,y,image,(760000,774000),(2802000,2807000))
fig,ax = plt.subplots(1,1)
h=ax.pcolorfast(newx,newy,newim,cmap=cm)
ax.set_aspect('equal')

In [ ]:
x.shape